# Heuristics vs Mathematical Optimization Using the Binary Paintshop Problem
- In car manufacturing one of the final production steps is painting.
- Multiple cars of different types (A to D) arrive in a given sequence at the paintshop.
 ![sequence](car_sequence.png)
- The cars have to be painted with a base coat that is either white or black (here referred to as red or blue).
- The demand for white and black colors for a given car type is also given.

This problem can be simplified to a minimal working example:
- In the sequence of cars arriving at the paintshop each vehicle type arrives exactly twice.
- One car of each vehicle type has to be painted white, the other one has to be painted black.

As changing colors requires time and produces waist, the goal is to minimize the number of color changes with respect to the constraint of coloring one car white and one black for each vehicle type.

This problem can be solved both heuristically or with a mathematical optimization approach.

In [ ]:
# Install dependencies
! pip install -q gamspy
#! gamspy install license ...
! gamspy install solver scip

In [ ]:
! pip install planqk-service-sdk

In [ ]:
import random
random.seed(16)

## Data

In [ ]:
sequence = ["A", "D", "E", "B", "A", "F", "C", "B", "C", "D", "E", "F"]
types = set(sequence)
n_types = len(types)
n_types

## Heuristic
- Start to color every vehicle type white
- Continue to use white as long as possible
- Than switch color until every car is painted

In [ ]:
changes = 0
colors = {"white": set(), "black": set()}
result = []


def paint_car(colors_dict, result_list, color, car_type):
    colors_dict[color].add(car_type)
    result_list.append(color)


current_color = "white"
for car in sequence:
    if car not in colors[current_color]:
        paint_car(colors, result, current_color, car)
    else:
        # change color
        changes += 1
        if current_color == "white":
            current_color = "black"
        else:
            current_color = "white"
        paint_car(colors, result, current_color, car)

print(result)
print("Number of changes:", changes)

## Mathematical Modeling

First we define the necessary sets and our variable $X_i$ which is a binary variable and has domain $i$.

In [ ]:
import gamspy as gp
from gamspy.math import sqr

# create container
m = gp.Container()

# create sets
i = gp.Set(m, "i", description="number in sequence")
j = gp.Set(m, "j", description="car type")
IJ = gp.Set(
    m,
    "IJ",
    domain=[i, j],
    records=[(i + 1, sequence[i]) for i in range(len(sequence))],
    domain_forwarding=True,
)

# create variables
X = m.addVariable("X", domain=[i], type="binary", description="color indicator")

We define our constraint 

$$\sum_{i: (i,j) \in \mathcal{IJ}} X_i = 1; \forall \ j \in \mathcal{J}$$

as an Equation. Since it holds for all $j$ the equation is in the domain $j$. Then we can directly use `gp.Sum()` to define the constraint.

The objective 
$$\sum_{i \in \mathcal{I} \hspace{0.75mm} | \hspace{0.75mm} i < |I|} (X_i - X_{i+1})^2$$

is defined as an expression.

In [ ]:
BlackOnce = gp.Equation(
    container=m,
    name="BlackOnce",
    domain=j,
    description="Ensure that each position i is painted black exactly once.",
)

BlackOnce[j] = gp.Sum(IJ[i, j], X[i]) == 1

obj = gp.Sum(i.where[gp.Ord(i) < gp.Card(i)], sqr(X[i] - X[i + 1]))

Finally, we assemble everything in our model. Since the objective is a quadratic function the model type is MIQCP (Mixed Integer Quadratically Constrained Program). Now we only need to solve it with a specified solver, here we pick CPLEX.

In [ ]:
paintshop = gp.Model(
    container=m,
    name="paintshop",
    equations=[BlackOnce],
    problem="MIQCP",
    sense=gp.Sense.MIN,
    objective=obj,
)

paintshop.solve(solver="CPLEX")

Now we can display the objective value, meaning the number of changes:

In [ ]:
opt_changes = paintshop.objective_value
opt_changes

Or also the value of X for each position:

In [ ]:
X.records["level"]

## Compare

In [ ]:
def display_colored_sequence(method, sequence, color_data):

    # Define ANSI color codes
    RED = "\033[91m"
    BLUE = "\033[94m"
    RESET = "\033[0m"

    colored_output = []

    if method.lower() == "optimization":
        # Dynamically assign red to the starting level
        color_map = {color_data[0]: RED, 1 - color_data[0]: BLUE}
        for i, char in enumerate(sequence):
            colored_output.append(color_map[color_data[i]])
            colored_output.append(char)

    elif method.lower() == "heuristic":
        for i, char in enumerate(sequence):
            # Assign red to 'white' and blue to anything else
            color = RED if color_data[i] == "white" else BLUE
            colored_output.append(color)
            colored_output.append(char)

    colored_output.append(RESET)

    print("The coloring sequence is:", "".join(colored_output))


### Optimization

In [ ]:
print(f"Using the optimization approach {round(opt_changes)} color changes are needed.")
display_colored_sequence("optimization", sequence, X.records["level"])

### Heuristic

In [ ]:
print(f"Using the heuristic approach {changes} color changes are needed.")
display_colored_sequence("heuristic", sequence, result)

## Transfer to the Real World - Multi Vehicle Paintshop Problem
The presented problem is a very easy and simplified version of the real problem where a given number of vehicles of different type arrive in a given sequence at the paint shop and a given share of each vehicle type has to be painted black and the rest white. However, the simplified version gives us a slight impression of how powerful mathematical optimization is. 
Solving the real (multi vehicle paint shop problem) is a more complicated version of the presented (binary paint shop problem).  

## Data

In [ ]:
sequence = random.choices(list(types), k=128)
demand_white = {t: random.randint(0, sequence.count(t)) for t in types}
demand_black = {t: sequence.count(t) - demand_white[t] for t in types}
demand_white

## Heuristic

In [ ]:
changes = 0
colors = {"white": dict(demand_white), "black": dict(demand_black)}
result = []


def paint_car(colors_dict, result_list, color, car_type):
    colors_dict[color][car_type] -= 1
    result_list.append(color)


current_color = "white"
for car in sequence:
    if colors[current_color][car] > 0:
        paint_car(colors, result, current_color, car)
    else:
        # change color
        changes += 1
        if current_color == "white":
            current_color = "black"
        else:
            current_color = "white"
        paint_car(colors, result, current_color, car)


print(f"Using the heuristic approach {changes} color changes are needed.")
display_colored_sequence("heuristic", sequence, result)

## Model

### What modifications do we need to make to the constraint?

So far: $$\sum_{i: (i,j) \in \mathcal{IJ}} X_i = 1; \forall \ j \in \mathcal{J}$$


Now: 

$$\sum_{i: (i,j) \in \mathcal{IJ}} X_i = ? \forall \ j \in \mathcal{J}$$

### What modifications do we need to make to the model implementation?

First need to update the records of the set to account for the longer sequence:

In [ ]:
IJ.setRecords([(i + 1, sequence[i]) for i in range(len(sequence))])

Then we need to introduce the new parameters $d_j^{black}$:

In [ ]:

# create parameters
black_demand = gp.Parameter(
    m,
    "black_demand",
    domain=[j],
    records=[(type, demand) for type, demand in demand_black.items()],
)


And define the corresponding constraints, e.g. Equations:

In [ ]:
MeetBlackDemand = gp.Equation(m, "MeetBlackDemand", domain=j)
MeetBlackDemand[j] = gp.Sum(IJ[i, j], X[i]) == ...

In [ ]:
multi_paintshop = gp.Model(
    m,
    "MultiPaintshop",
    equations=[MeetBlackDemand],
    problem="MIQCP",
    sense=gp.Sense.MIN,
    objective=obj,
)

multi_paintshop.solve(solver="cplex")

In [ ]:
multi_paintshop.solve(solver="scip")

In [ ]:
# multi_paintshop.solve(solver="shot")

In [ ]:
opt_changes = multi_paintshop.objective_value
opt_changes

## Compare

### Mathematical Model

In [ ]:
# save coloring to display later
if round(X.records["level"][0], 1) == 0:
    colors = {0: "\033[91m", 1: "\033[94m"}
else:
    colors = {1: "\033[91m", 0: "\033[94m"}

opt_coloring_multi_car = []
for x in range(len(sequence)):
    opt_coloring_multi_car.append(colors[round(X.records["level"][x], 1)])
    opt_coloring_multi_car.append(sequence[x])

opt_coloring_multi_car = "".join(opt_coloring_multi_car)

print(f"Using the optimization approach {round(opt_changes)} color changes are needed.")
display_colored_sequence("optimization", sequence, X.records["level"])

### Heuristic

In [ ]:
print(f"Using the heuristic approach {changes} color changes are needed.")
display_colored_sequence("heuristic", sequence, result)

In [ ]:
! pip install planqk-service-sdk
! git clone https://github.com/GAMS-dev/QUBO.git && cd QUBO/gamspy_qubo && pip install . && pip install -r requirements.txt

In [ ]:
import json
import numpy as np
from gamspy_qubo import Qubo
from planqk.service.client import PlanqkServiceClient


types = set(["A", "B"])
n_types = len(types)

# Multi Vehicle Paintshop Problem

sequence = random.choices(list(types), k=6)
print(sequence)
demand_white = {t: random.randint(0, sequence.count(t)) for t in types}
demand_black = {t: sequence.count(t) - demand_white[t] for t in types}

IJ.setRecords([(i + 1, sequence[i]) for i in range(len(sequence))])
black_demand.setRecords([(type, demand) for type, demand in demand_black.items()])

paintshop = gp.Model(
    m,
    name="paintshop",
    problem="MIQCP",
    equations=[MeetBlackDemand],
    sense=gp.Sense.MIN,
    objective=obj,
)
paintshop.solve()
print(X.records)

paintshop_qubo = Qubo(paintshop, name="paintshop_qubo", penalty=10)
paintshop_qubo.solve()

qubo_matrix = paintshop_qubo.qubo

rows_idx, cols_idx = np.triu_indices(qubo_matrix.shape[0])

values = qubo_matrix[rows_idx, cols_idx]

final_qubo = {}
for r, c, v in zip(rows_idx, cols_idx, values):
    if r == c:
        final_qubo[f"({r},)"] = v
    else:
        final_qubo[f"({r},{c})"] = v

from google.colab import userdata

consumer_key = userdata.get("PlanqkConsumerKey")
consumer_secret = userdata.get("PlanqkConsumerSecret")

free_dcqo_endpoint = "https://gateway.platform.planqk.de/kipu-quantum/kipu-digitized-counterdiabatic-quantum-optimization---dcqo/1.0.0"
client = PlanqkServiceClient(free_dcqo_endpoint, consumer_key, consumer_secret)

data = {
    "optimization": {
        "coefficients": final_qubo,
        "annealing_time": 0.7,
        "trotter_steps": 2,
        "mode": "CD",
    }
}

params = {"backend": "azure.ionq.simulator", "shots": 1024}

job = client.run(request={"data": data, "params": params})
job_id = job.id

result = client.get_service_execution(job_id)
counts = result.result().dict()["counts"]
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
json_string = json.dumps(counts)

print(json_string)